# 영화 리뷰 데이터 분류하기

## 데이터 불러오기

In [1]:
from nltk.tokenize import word_tokenize
import nltk

In [2]:
def load_data(file_path):
    #file_path에 있는 데이터를 읽어 옴
    
    train = []
    
    count = 0
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            if count == 500: break     # 500개만 가져옴
            line = line.strip()
            id, doc, label = line.split('\t')
            if label == '1': label = 'pos'
            elif label == '0': label = 'neg'
            train.append((doc,label))
            count += 1
            
    return train

In [3]:
train = load_data('ratings_train.txt')
print(train[:5])

[('document', 'label'), ('아 더빙.. 진짜 짜증나네요 목소리', 'neg'), ('흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', 'pos'), ('너무재밓었다그래서보는것을추천한다', 'neg'), ('교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', 'neg')]


In [4]:
train = train[1:]
print(train[:5])

[('아 더빙.. 진짜 짜증나네요 목소리', 'neg'), ('흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', 'pos'), ('너무재밓었다그래서보는것을추천한다', 'neg'), ('교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', 'neg'), ('사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', 'pos')]


## 단어 집합 만들기

In [5]:
from konlpy.tag import Okt
okt = Okt()

In [7]:
def pos_tokenize(raw_sent):
    pos_sent = []
    
    #raw_sent: 사과가 좋아
    sent = okt.pos(raw_sent, norm=True, stem=True)
    #sent: [('사과', 'Noun'),('가','Josa'),('좋다','Adjective')]
    
    for tup in sent: 
        word, tag = tup[0], tup[1]    # tup: ('사과', 'Noun')
        word_tag = word + '/' + tag  # word_tag: '사과/Noun'
        pos_sent.append(word_tag)
        
    return ' '.join(pos_sent)

In [8]:
all_words = set()

for tup in train:
    sent, label = tup[0], tup[1]  #ex) sent = 'l like you', label = 'pos'
    sent = pos_tokenize(sent)
    words = word_tokenize(sent)
    for word in words:
        all_words.add(word)

In [9]:
for tup in train:
        sent, label = tup[0], tup[1]
        sent = pos_tokenize(sent)
        words = word_tokenize(sent)
        for word in words:
            all_words.add(word)

In [48]:
train_features = []

for tup in train:
    sent, label = tup[0], tup[1]
    sent = pos_tokenize(sent)
    words = word_tokenize(sent)
    tmp = {set_word: (set_word in words) for set_word in all_words} # set_word가 words에 있는지 여부 판별
    sent_tup = (tmp, label)
    train_features.append(sent_tup)
    
for i in range(len(train_features)):
    print(train_features[i])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## 분류기 학습시키기

In [49]:
classifier = nltk.NaiveBayesClassifier.train(train_features)
classifier.show_most_informative_features()

Most Informative Features
                       ; = True              neg : pos    =      8.2 : 1.0
          재미없다/Adjective = True              neg : pos    =      8.2 : 1.0
                주인공/Noun = True              neg : pos    =      7.5 : 1.0
                 최고/Noun = True              pos : neg    =      6.9 : 1.0
                  뭐/Noun = True              neg : pos    =      6.8 : 1.0
           재밌다/Adjective = True              pos : neg    =      6.5 : 1.0
                 내용/Noun = True              neg : pos    =      6.2 : 1.0
       ㅡㅡ/KoreanParticle = True              neg : pos    =      6.1 : 1.0
                스토리/Noun = True              neg : pos    =      6.1 : 1.0
                 다시/Noun = True              pos : neg    =      5.9 : 1.0


In [55]:
test_sent = '재미없다'
test_sent = pos_tokenize(test_sent)
words = word_tokenize(test_sent)
test_feature = {set_word: (set_word in words) for set_word in all_words}

In [56]:
classifier.classify(test_feature)

'neg'

# 테스트 해보기

In [27]:
import urllib
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt","rating_test.txt")

('rating_test.txt', <http.client.HTTPMessage at 0x20414688370>)

In [57]:
# ratings_test.txt를 처리하는 부분 작성
test = load_data('ratings_test.txt') # load_data('ratings_test.txt')
test

[('document', 'label'),
 ('굳 ㅋ', 'pos'),
 ('GDNTOPCLASSINTHECLUB', 'neg'),
 ('뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아', 'neg'),
 ('지루하지는 않은데 완전 막장임... 돈주고 보기에는....', 'neg'),
 ('3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??', 'neg'),
 ('음악이 주가 된, 최고의 음악영화', 'pos'),
 ('진정한 쓰레기', 'neg'),
 ('마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다', 'neg'),
 ('갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다',
  'neg'),
 ('이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..', 'pos'),
 ('괜찮네요오랜만포켓몬스터잼밌어요', 'pos'),
 ('한국독립영화의 한계 그렇게 아버지가 된다와 비교됨', 'neg'),
 ('청춘은 아름답다 그 아름다움은 이성을 흔들어 놓는다. 찰나의 아름다움을 잘 포착한 섬세하고 아름다운 수채화같은 퀴어영화이다.',
  'pos'),
 ('눈에 보이는 반전이었지만 영화의 흡인력은 사라지지 않았다.', 'pos'),
 ('"""스토리, 연출, 연기, 비주얼 등 영화의 기본 조차 안된 영화에 무슨 평을 해. 이런 영화 찍고도 김문옥 감독은 """"내가 영화 경력이 몇OO인데 조무래기들이 내 영화를 평론해?"""" 같은 마인드에 빠져있겠지?"""',
  'neg'),
 ('소위 ㅈ문가라는 평점은 뭐냐?', 'pos'),
 ('최고!!!!!!!!!!!!!!!!', 'pos'),
 ('발연기 도저히 못보겠다 진짜 이렇게 연기를 못할거라곤 상상도 못했네', 'neg'),
 ('나이스', 'pos'),
 ('별 재미도없는거 우려먹어 .... 챔프에서 방송 몇

In [58]:
test_sent = test[1][0] # 첫 번째 문장 입력 # '영화가 재미없다 ㅡㅡ'

test_sent = pos_tokenize(test_sent)
print(test_sent)

words = word_tokenize(test_sent)
test_feature = {set_word: (set_word in words) for set_word in all_words}

굳다/Adjective ㅋ/KoreanParticle


In [59]:
classifier.classify(test_feature)

'pos'